# OMOP to Drugs For Treatments

As in from IDEA4RC to OMOP, this is not clear yet and it is hard to understand the mapping in: https://docs.google.com/spreadsheets/d/1Vw1Dr2K4oG__cDQTutGaJhZvGUvQTLwc4qWreP6qMSs/edit?gid=634754097#gid=634754097 

In [ ]:
import mysql.connector
import pandas as pd
import psycopg2
import random
import numpy as np
from datetime import datetime, timedelta

In [ ]:
conn = psycopg2.connect(
    dbname="omopdb",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5432"
)

curOMOP = conn.cursor()

config = {
    'user': 'user', 
    'password': 'password',
    'host': '127.0.0.1',
    'database': 'idea4rc_dm',
    'raise_on_warnings': True
}

conn2 = mysql.connector.connect(**config)
curIDEA = conn2.cursor()

In [ ]:
getProcedureOccurrenceType= """
    SELECT procedure_concept_id FROM procedure_occurrence
    procedure_occurrence_id IN (SELECT event_id FROM episode_event WHERE episode_id=%s);
"""

queryAllExposures = """
    SELECT person_id, drug_concept_id, drug_exposure_start_date, drug_exposure_end_date, ep.episode_id FROM drug_exposure
    INNER JOIN episode_event ep ON ep.event_id=drug_exposure.drug_exposure_id
    WHERE episode_event_field_concept_id=1147094;
"""

queryGetOriginalEpisode= """
    SELECT episode_start_date, episode_end_date FROM episode_event WHERE id=%s;
"""

getParentId= """
    SELECT parent_id FROM episode WHERE episode_id=%s;
"""

curOMOP.execute(queryAllExposures)
exposures=curOMOP.fetchall()


for exposure in exposures:
    curOMOP.execute(getProcedureOccurrenceType, (exposure[0], exposure[2, exposure[3]]))
    type = curOMOP.fetchone()
    table=None
    if type in (912065,40310107,4273629):
        table="systemic_treatment"
    elif type in (4137449,9177):
        table="other_local_treatment"
    elif type == 4019656:
        table="isolated_limb_perfusion"
    
    sqlGetForeignKey = """
        SELECT id FROM {table} WHERE start_date=%s AND end_date=%s
        AND episode_event IN (SELECT id FROM episode_event WHERE patient=%s);
    """
    curOMOP.execute(getParentId, (exposure[4]))
    parentId = curOMOP.fetchone()
    curOMOP.execute(queryGetOriginalEpisode, (parentId))
    episode = curOMOP.fetchone()
    curIDEA.execute(sqlGetForeignKey.format(table=table), (episode[0], episode[1], exposure[0]))
    foreignKey = curIDEA.fetchone()
    insertQuery = """
        INSERT INTO drugs_for_treatments ({table},drug)
        VALUES (%s, %s);
    """
    curIDEA.execute(insertQuery.format(table=table), (foreignKey, exposure[1]))
    conn2.commit()